# Library

In [2]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import pandas as pd
import datetime, random, math
from catboost import CatBoostClassifier
import lightgbm as lgb
from time import time
from tqdm import tqdm
from collections import Counter
from scipy import stats
import gc, pickle
import ast

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error, log_loss
from sklearn.linear_model import Ridge,Lasso, BayesianRidge
from sklearn.svm import LinearSVR
from sklearn.preprocessing import minmax_scale
from sklearn.cluster import KMeans
import optuna
%matplotlib inline

# Preprocessing

In [3]:
def create_is_sell_data(sell_prices_df, calendar_df, train_df):
    train_df.index = train_df['id']
    sell_prices_df['id'] = sell_prices_df['item_id'].astype('str')+'_'+sell_prices_df['store_id']+'_evaluation'
    sell_prices_data = sell_prices_df[sell_prices_df.wm_yr_wk.isin(calendar_df.wm_yr_wk.unique())]
    sell_prices_data.reset_index(drop=True, inplace=True)
    tmp = sell_prices_data.groupby(['id'])[['wm_yr_wk', 'sell_price']].apply(
        lambda x: x.set_index('wm_yr_wk')['sell_price'].to_dict()
    ).to_dict()
    d = calendar_df.d
    wm_yr_wk = calendar_df.wm_yr_wk
    price_data = {}
    for col in tqdm(train_df.id.unique()):
        price_data[col] = wm_yr_wk.map(tmp[col])
    price_data = pd.DataFrame(price_data)
    price_data.index = d
    is_sell = price_data.notnull().astype(float).T
    price_data = price_data.fillna(0)
    
    is_sell.index=train_df.id
    train_df.index=train_df.id
    is_sell = pd.concat([
        train_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']], is_sell
    ], axis=1)
    price_data = pd.concat([
        train_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']], price_data.T  
    ], axis=1)
    
    return price_data, is_sell

def sort_d_cols(d_cols):
    d_cols = [int(d.replace('d_','')) for d in d_cols]
    d_cols = sorted(d_cols)
    d_cols = [f'd_{d}' for d in d_cols]
    return d_cols


path='/Users/abcdm/Downloads/m5-forecasting-accuracy/'
train = pd.read_csv(path+'sales_train_evaluation.csv')
calendar = pd.read_csv(path+'calendar.csv')
price = pd.read_csv(path+'sell_prices.csv')
price_data, is_sell = create_is_sell_data(price, calendar, train)
d_cols = [f'd_{i+1}' for i in range(1969)]
train = train.reindex(
    columns=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']+d_cols
)
train = train.set_index('id', drop=False)
train = pd.concat([
    train[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']],
    train[d_cols]*price_data[d_cols]
], axis=1)


def sale_by_group(ID, data, train, trn_d_cols, group='state_id'):
    for _id in train[group].unique():
        f = train[train[group]==_id].groupby(ID)[trn_d_cols].sum(min_count=1).T
        f = f.stack(dropna=False).reset_index().rename(columns={0:f'TARGET_{_id}', 'level_0':'d'})
        data = pd.merge(data, f, on=['d', ID])
    return data

def cnt_by_group(ID, data, price_data, trn_d_cols, group='state_id'):
    for _id in price_data[group].unique():
        f = price_data[price_data[group]==_id].replace(0, np.nan).groupby(ID)[trn_d_cols].count().T
        f = f.stack(dropna=False).reset_index().rename(columns={0:f'cnt_{_id}', 'level_0':'d'})
        data = pd.merge(data, f, on=['d', ID])
    return data


def fe(ID,data, log=False):
    target_cols = [col for col in data.columns if 'TARGET' in col]
    for target_col in target_cols:
        if log:
            data[target_col] = np.log1p(data[target_col])
        for win in [7,28]:
            agg = {'mean'}
            data_2 = data.groupby(ID)[target_col].apply(
                lambda x: x.rolling(win, min_periods=1).agg(agg)
            )
            for col in data_2.columns:
                data[f'roll{win}_{col}_{target_col}'] = data_2[col]
        #data[f'roll28_mean_{target_col}_lag28'] = data.groupby(ID)[f'roll28_mean_{target_col}'].shift(28)
        data[f'roll28_mean_{target_col}_lag56'] = data.groupby(ID)[f'roll28_mean_{target_col}'].shift(56)
        data[f'roll28_mean_{target_col}_lag84'] = data.groupby(ID)[f'roll28_mean_{target_col}'].shift(84)
        
        for i in range(0,10,2):
                data[f'shift{7*(i+1)}_{target_col}'] = data.groupby(ID)[target_col].shift(7*i)
                
        data[f'shift_diff_{target_col}']=0
        for i in range(4):
                data[f'shift_diff_{target_col}'] += data.groupby(ID)[target_col].diff(7*i)/4

    data_2 = data.groupby(ID)['TARGET'].apply(
                lambda x: x.rolling(4, min_periods=1).mean().rolling(21, min_periods=1).agg({'std'})
            )
    for col in data_2.columns:
                data[f'smth_roll28_{col}_TARGET'] = data_2[col]
    
    del_f = [col for col in target_cols if col!='TARGET']
    data.drop(columns=del_f, inplace=True)
    
    return data


def select_near_event(x, event_name):
    z = ''
    for y in x:
        if y in event_name:
            z+=y+'_'
    if len(z)==0:
        return np.nan
    else:
        return z
    
    
calendar['qaurter'] = pd.to_datetime(calendar['date']).dt.day.apply(lambda x: x//7)

event_name = ['SuperBowl', 'ValentinesDay', 'PresidentsDay', 'LentStart', 'LentWeek2', 'StPatricksDay', 'Purim End', 
              'OrthodoxEaster', 'Pesach End', 'Cinco De Mayo', "Mother's day", 'MemorialDay', 'NBAFinalsStart', 'NBAFinalsEnd',
              "Father's day", 'IndependenceDay', 'Ramadan starts', 'Eid al-Fitr', 'LaborDay', 'ColumbusDay', 'Halloween', 
              'EidAlAdha', 'VeteransDay', 'Thanksgiving', 'Christmas', 'Chanukah End', 'NewYear', 'OrthodoxChristmas', 
              'MartinLutherKingDay', 'Easter']
event_type = ['Sporting', 'Cultural', 'National', 'Religious']
event_names = {'event_name_1':event_name, 'event_type_1':event_type}
for event, event_name in event_names.items():
    calendar[f'new_{event}']=''
    for i in range(-1,-8,-1):
        calendar[f'new_{event}'] += calendar[event].shift(i).astype(str)+'|'
    calendar[f'new_{event}'] = calendar[f'new_{event}'].apply(lambda x: x.split('|'))
    calendar[f'new_{event}'] = calendar[f'new_{event}'].apply(lambda x: select_near_event(x, event_name))
    
    
def make_data_dept_cat_id(ID, trn_d_cols, train, price_data, calendar, log=True):
    data = train.groupby(ID)[trn_d_cols].sum(min_count=1).T
    data = data.stack(dropna=False).reset_index().rename(columns={0:'TARGET', 'level_0':'d'})
    f = price_data.replace(0,np.nan).groupby(ID)[
        trn_d_cols
    ].count().stack(dropna=False).reset_index().rename(columns={0:'cnt', 'level_1':'d', 'level_0':'d'})
    data = pd.merge(data, f, on=['d', ID])
    data = sale_by_group(ID, data, train, trn_d_cols)

    data = cnt_by_group(ID, data, price_data, trn_d_cols)
    data = fe(ID, data, log=log)

    for key, value in calendar.set_index('d')[['qaurter','wday','month','year','new_event_name_1', 'new_event_type_1','event_name_1','event_type_1','snap_CA','snap_WI','snap_TX']].items():
        data[key] = data.d.map(value)
        if data[key].dtypes==object:
            data[key] = pd.factorize(data[key])[0]

        if key in ['event_name_1','event_type_1','snap_CA','snap_WI','snap_TX']:
            for i in [ -1,1]:
                data[f'{key}_{i}'] = data.groupby(ID)[key].shift(i)

    data[f'f_{ID}'] = pd.factorize(data[ID])[0]
    data = data[data.d.isin(trn_d_cols[28:])]
    return data

def make_data_all_id(ID, trn_d_cols, train, price_data, calendar):
    data = train.groupby(ID)[trn_d_cols].sum(min_count=1).T
    data = data.stack(dropna=False).reset_index().rename(columns={0:'TARGET', 'level_0':'d'})
    f = price_data.replace(0,np.nan).groupby(ID)[
        trn_d_cols
    ].count().stack(dropna=False).reset_index().rename(columns={0:'cnt', 'level_1':'d', 'level_0':'d'})
    data = pd.merge(data, f, on=['d', ID])
    data = sale_by_group(ID, data, train, trn_d_cols, group='state_id')

    data = cnt_by_group(ID, data, price_data, trn_d_cols)
    data = fe(ID, data, log=False)

    for key, value in calendar.set_index('d')[['qaurter','wday','month','year','new_event_name_1', 'new_event_type_1','event_name_1','event_type_1','snap_CA','snap_WI','snap_TX']].items():
        data[key] = data.d.map(value)
        if data[key].dtypes==object:
            data[key] = pd.factorize(data[key])[0]

        if key in ['event_name_1','event_type_1','snap_CA','snap_WI','snap_TX']:
            for i in [ -1,1]:
                data[f'{key}_{i}'] = data.groupby(ID)[key].shift(i)

    data[f'f_{ID}'] = pd.factorize(data[ID])[0]
    data = data[data.d.isin(trn_d_cols[28:])]
    return data
def make_data_state_id_store_id(ID, trn_d_cols, train, price_data, calendar, log=True):
    
    data = train.groupby(ID)[trn_d_cols].sum(min_count=1).T
    data = data.stack(dropna=False).reset_index().rename(columns={0:'TARGET', 'level_0':'d'})
    f = price_data.replace(0,np.nan).groupby(ID)[
        trn_d_cols
    ].count().stack(dropna=False).reset_index().rename(columns={0:'cnt', 'level_1':'d', 'level_0':'d'})
    data = pd.merge(data, f, on=['d', ID])
    
    group='cat_id'
    for _id in train[group].unique():
        f = train[train[group]==_id].groupby(ID)[trn_d_cols].sum(min_count=1).T
        f = f.stack(dropna=False).reset_index().rename(columns={0:f'TARGET_{_id}', 'level_0':'d'})
        data = pd.merge(data, f, on=['d', ID])
    
    for _id in price_data[group].unique():
        f = price_data[price_data[group]==_id].replace(0, np.nan).groupby(ID)[trn_d_cols].count().T
        f = f.stack(dropna=False).reset_index().rename(columns={0:f'cnt_{_id}', 'level_0':'d'})
        data = pd.merge(data, f, on=['d', ID])
    
    
    data = fe(ID, data, log=log)

    for key, value in calendar.set_index('d')[['qaurter','wday','month','year','new_event_name_1', 'new_event_type_1','event_name_1','event_type_1']].items():
        data[key] = data.d.map(value)
        if data[key].dtypes==object:
            data[key] = pd.factorize(data[key])[0]

        if key in ['event_name_1','event_type_1']:
            for i in [ -1,1]:
                data[f'{key}_{i}'] = data.groupby(ID)[key].shift(i)
                
    data['snap'] = 0
    for key, value in calendar.set_index('d')[['snap_CA','snap_WI','snap_TX']].items():
        state = key.replace('snap_', '')
        data.loc[data[ID].str.contains(state),'snap'] = data.loc[data[ID].str.contains(state),'d'].map(value)
    for i in [ -1,1]:
        data[f'snap_{i}'] = data.groupby(ID)['snap'].shift(i)
    
    data[f'f_{ID}'] = pd.factorize(data[ID])[0]
    data = data[data.d.isin(trn_d_cols[28:])]
    return data

def make_data_2_id(ID, trn_d_cols, train, price_data, calendar, log=True):
    
    data = train.groupby(ID)[trn_d_cols].sum(min_count=1).T
    data = data.stack(dropna=False).reset_index().rename(columns={0:'TARGET', 'level_0':'d'})
    f = price_data.replace(0,np.nan).groupby(ID)[
        trn_d_cols
    ].count().stack(dropna=False).reset_index().rename(columns={0:'cnt', 'level_1':'d', 'level_0':'d'})
    data = pd.merge(data, f, on=['d', ID])
    
    data = fe(ID, data, log=log)

    for key, value in calendar.set_index('d')[['qaurter','wday','month','year','new_event_name_1', 'new_event_type_1','event_name_1','event_type_1']].items():
        data[key] = data.d.map(value)
        if data[key].dtypes==object:
            data[key] = pd.factorize(data[key])[0]

        if key in ['event_name_1','event_type_1']:
            for i in [ -1,1]:
                data[f'{key}_{i}'] = data.groupby(ID)[key].shift(i)
                
    data['snap'] = 0
    for key, value in calendar.set_index('d')[['snap_CA','snap_WI','snap_TX']].items():
        state = key.replace('snap_', '')
        data.loc[data[ID].str.contains(state),'snap'] = data.loc[data[ID].str.contains(state),'d'].map(value)
    for i in [ -1,1]:
        data[f'snap_{i}'] = data.groupby(ID)['snap'].shift(i)
    
    data[f'f_{ID}'] = pd.factorize(data[ID])[0]
    data = data[data.d.isin(trn_d_cols[28:])]
    return data

100%|██████████| 30490/30490 [00:11<00:00, 2770.17it/s]


# lgb model utils

In [16]:
class Optimize_Optuna():
    def __init__(self, data, ID):
        days = data.d.unique().tolist()
        days = sort_d_cols(days)
        self.trn_days = days[:-28]
        val_days = days[-28:]
        
        self.X = data[data.d.isin(self.trn_days)]
        self.ID = ID
        
        self.shift_cols = [col for col in data.columns if 'shift' in col]
        self.roll_cols = [col for col in data.columns if 'roll' in col]
        cat_cols = [col for col in data.columns if (not 'shift' in col) and (not 'roll' in col)]
        self.cat_cols = [col for col in cat_cols if not col in [ID, 'd', 'TARGET']]
        self.features=self.cat_cols+self.shift_cols+self.roll_cols

    def objective(self, trial):
        PARAMS = {
            'boosting_type': 'gbdt','objective': 'rmse', 'metric': 'rmse','random_state':2020,'max_bin': 100,
            'n_estimators': 1400,'boost_from_average': False,'verbose': -1,
            
            'subsample': trial.suggest_uniform('subsample', 0.4, 0.9),
            'subsample_freq': trial.suggest_uniform('subsample_freq', 0.4, 0.9),
            'learning_rate': trial.suggest_loguniform('learning_rate', 0.03, 0.5),
            'num_leaves': trial.suggest_int('num_leaves', 2**6, 2**7),
            'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
            'lambda_l2': trial.suggest_loguniform('lambda_l2', 0.1, 10.0)
        }
        score = 0

        data = self.X.copy()
        for i in range(28):
            
            if i%7==0:
                data[self.shift_cols] = data.groupby(self.ID)[self.shift_cols].shift(7)
            data[self.roll_cols] = data.groupby(self.ID)[self.roll_cols].shift(1)
            
            split = data[data.d.isin(self.trn_days[-500:])]['TARGET']
            split = split.mean()-3.*split.std()
            X = data[data.TARGET>split]
            X.reset_index(drop=True, inplace=True)
            k = StratifiedKFold(n_splits=5, random_state=2020, shuffle=True)
            
            y = (200*minmax_scale(X['TARGET'])).astype(int)
            t_score=0
            for trn, val in k.split(X,y=y, groups=y):
                train_set = lgb.Dataset(X.loc[trn,self.features], X.loc[trn,'TARGET'])
                val_set = lgb.Dataset(X.loc[val,self.features], X.loc[val,'TARGET'])

                model = lgb.train(train_set=train_set, valid_sets=[train_set, val_set], params=PARAMS, num_boost_round=3000, 
                                  early_stopping_rounds=100, verbose_eval=10000)
                
                preds = model.predict(X.loc[val,self.features])
                
                preds = np.e**(preds)-1
                true_y = np.e**(X.loc[val,'TARGET'])-1
                
                #true_y = X.loc[val,'TARGET']
                t_score+=np.sqrt(mean_squared_error(preds, true_y))/k.n_splits
            score+=t_score/28
        return score

In [22]:
def all_flow(study, n_trials):
    d_cols = [f'd_{i+1}' for i in range(1969)]
    d_cols = d_cols[:-28]
    trn_d_cols = d_cols[-730:]
    ID='dept_idXstore_id'
    #ID='all_id'

    train['dept_idXstore_id'] = train['dept_id'].astype(str)+'X'+train['store_id'].astype(str)
    price_data['dept_idXstore_id'] = train['dept_id'].astype(str)+'X'+train['store_id'].astype(str)
    train['all_id'] = 'all_id'
    price_data['all_id'] = 'all_id'
    
    #data = make_data_all_id(ID, trn_d_cols, train, price_data, calendar)
    
    data = make_data_2_id(ID, trn_d_cols, train, price_data, calendar, log=True)
    
    #data = make_data_dept_cat_id(ID=ID, trn_d_cols=trn_d_cols, train=train,
     #                                price_data=price_data, calendar=calendar, log=True)

    optimize_optuna = Optimize_Optuna(data, ID)

    study.optimize(optimize_optuna.objective, n_trials=n_trials)
    return study

In [23]:
%%time
#with open(f'../input/fork-of-fork-of-optuna-of-make-data-create-m5-all/study_24times.pickle', 'rb') as f:
 #   study = pickle.load(f)

study = optuna.create_study(direction='minimize')
n_trials=20
study = all_flow(study, n_trials)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	training's rmse: 0.135296	valid_1's rmse: 0.191779
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[76]	training's rmse: 0.124764	valid_1's rmse: 0.185374
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[33]	training's rmse: 0.151614	valid_1's rmse: 0.191114
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	training's rmse: 0.141531	valid_1's rmse: 0.184415
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[48]	training's rmse: 0.14132	valid_1's rmse: 0.185446
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[80]	training's rmse: 0.123233	valid_1's rmse: 0.190833
Training until validation scores don't improve for 100 rounds
Early stopping, best 

[87]	training's rmse: 0.124451	valid_1's rmse: 0.186408
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[81]	training's rmse: 0.126606	valid_1's rmse: 0.189375
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[132]	training's rmse: 0.104691	valid_1's rmse: 0.193195
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[71]	training's rmse: 0.131171	valid_1's rmse: 0.188721
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[131]	training's rmse: 0.105048	valid_1's rmse: 0.191996
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	training's rmse: 0.140921	valid_1's rmse: 0.185225
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	training's rmse: 0.136937	valid_1's rmse: 0.187663
Training until validatio

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	training's rmse: 0.140685	valid_1's rmse: 0.192356
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[96]	training's rmse: 0.121987	valid_1's rmse: 0.185971
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[67]	training's rmse: 0.135141	valid_1's rmse: 0.188967
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[131]	training's rmse: 0.106155	valid_1's rmse: 0.194151
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[69]	training's rmse: 0.132864	valid_1's rmse: 0.191907
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[55]	training's rmse: 0.141185	valid_1's rmse: 0.193081
Training until validation scores don't improve for 100 rounds
Early stopping, bes

[I 2020-06-21 13:23:11,377] Finished trial#0 with value: 259.18409082202754 with parameters: {'subsample': 0.833893939750052, 'subsample_freq': 0.44484050991164464, 'learning_rate': 0.23389140425658878, 'num_leaves': 127, 'feature_fraction': 0.5309236883917332, 'bagging_freq': 4, 'min_child_samples': 9, 'lambda_l2': 0.10341719131629068}. Best is trial#0 with value: 259.18409082202754.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	training's rmse: 0.153826	valid_1's rmse: 0.195227
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	training's rmse: 0.158496	valid_1's rmse: 0.189122
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[44]	training's rmse: 0.149645	valid_1's rmse: 0.193032
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[61]	training's rmse: 0.141641	valid_1's rmse: 0.187975
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[46]	training's rmse: 0.1505	valid_1's rmse: 0.188407
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	training's rmse: 0.164139	valid_1's rmse: 0.196709
Training until validation scores don't improve for 100 rounds
Early stopping, best i

...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[624]	training's rmse: 0.124591	valid_1's rmse: 0.185616
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[811]	training's rmse: 0.115514	valid_1's rmse: 0.180078
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[742]	training's rmse: 0.118689	valid_1's rmse: 0.184731
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1026]	training's rmse: 0.107141	valid_1's rmse: 0.177343
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[719]	training's rmse: 0.121056	valid_1's rmse: 0.179615
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[874]	training's rmse: 0.113022	valid_1's rmse: 0.185808
Training until validation scores don't improve for 100 rounds
Early stoppin

[1400]	training's rmse: 0.0950798	valid_1's rmse: 0.182461
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1212]	training's rmse: 0.101345	valid_1's rmse: 0.184442
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0960882	valid_1's rmse: 0.177855
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0959543	valid_1's rmse: 0.182244
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0944704	valid_1's rmse: 0.186245
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0955471	valid_1's rmse: 0.182712
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1193]	traini

[821]	training's rmse: 0.118018	valid_1's rmse: 0.185115
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0961194	valid_1's rmse: 0.177175
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0960199	valid_1's rmse: 0.181057
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0955045	valid_1's rmse: 0.186978
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400]	training's rmse: 0.0961025	valid_1's rmse: 0.183952
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1004]	training's rmse: 0.1104	valid_1's rmse: 0.18402
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1400

[I 2020-06-21 14:40:48,817] Finished trial#19 with value: 238.20964337524825 with parameters: {'subsample': 0.767911185577838, 'subsample_freq': 0.7287605276865973, 'learning_rate': 0.04115377841371611, 'num_leaves': 82, 'feature_fraction': 0.5709048571103937, 'bagging_freq': 1, 'min_child_samples': 17, 'lambda_l2': 0.45656192806764545}. Best is trial#19 with value: 238.20964337524825.


Wall time: 1h 18min 58s


In [28]:
{'subsample': 0.767911185577838, 'subsample_freq': 0.7287605276865973, 'learning_rate': 0.04115377841371611, 'num_leaves': 82, 'feature_fraction': 0.5709048571103937, 'bagging_freq': 1, 'min_child_samples': 17, 'lambda_l2': 0.45656192806764545}

{'subsample': 0.767911185577838,
 'subsample_freq': 0.7287605276865973,
 'learning_rate': 0.04115377841371611,
 'num_leaves': 82,
 'feature_fraction': 0.5709048571103937,
 'bagging_freq': 1,
 'min_child_samples': 17,
 'lambda_l2': 0.45656192806764545}

In [24]:
print(f"""
Number of finished trials :     {len(study.trials)}
Best value                :  {study.best_value :.3f}
""")

print('Best trial:', study.best_trial.params)


Number of finished trials :     20
Best value                :  238.210

Best trial: {'subsample': 0.767911185577838, 'subsample_freq': 0.7287605276865973, 'learning_rate': 0.04115377841371611, 'num_leaves': 82, 'feature_fraction': 0.5709048571103937, 'bagging_freq': 1, 'min_child_samples': 17, 'lambda_l2': 0.45656192806764545}


In [29]:
def all_flow(study, n_trials):
    d_cols = [f'd_{i+1}' for i in range(1969)]
    d_cols = d_cols[:-28]
    trn_d_cols = d_cols[-730:]
    ID='dept_id'
    #ID='all_id'

    train['dept_idXstore_id'] = train['dept_id'].astype(str)+'X'+train['store_id'].astype(str)
    price_data['dept_idXstore_id'] = train['dept_id'].astype(str)+'X'+train['store_id'].astype(str)
    train['all_id'] = 'all_id'
    price_data['all_id'] = 'all_id'
    
    #data = make_data_all_id(ID, trn_d_cols, train, price_data, calendar)
    
    #data = make_data_2_id(ID, trn_d_cols, train, price_data, calendar, log=True)
    
    data = make_data_dept_cat_id(ID=ID, trn_d_cols=trn_d_cols, train=train,
                                 price_data=price_data, calendar=calendar, log=True)

    optimize_optuna = Optimize_Optuna(data, ID)

    study.optimize(optimize_optuna.objective, n_trials=n_trials)
    return study

In [30]:
study = optuna.create_study(direction='minimize')
n_trials=20
study = all_flow(study, n_trials)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[156]	training's rmse: 0.023546	valid_1's rmse: 0.0941769
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1399]	training's rmse: 0.00117934	valid_1's rmse: 0.0926883
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[966]	training's rmse: 0.00323156	valid_1's rmse: 0.0904806
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[304]	training's rmse: 0.00839411	valid_1's rmse: 0.0873831
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[280]	training's rmse: 0.00991537	valid_1's rmse: 0.0940589
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[348]	training's rmse: 0.00678121	valid_1's rmse: 0.0920489
Training until validation scores don't improve

[516]	training's rmse: 0.00527145	valid_1's rmse: 0.090469
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[1395]	training's rmse: 0.00337464	valid_1's rmse: 0.0835759
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[334]	training's rmse: 0.00678906	valid_1's rmse: 0.0896324
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[340]	training's rmse: 0.00649498	valid_1's rmse: 0.0922187
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[329]	training's rmse: 0.00685681	valid_1's rmse: 0.094155
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[263]	training's rmse: 0.00887028	valid_1's rmse: 0.0906875
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1164]	training's rmse: 0.00380755	valid_1's r

[322]	training's rmse: 0.00786738	valid_1's rmse: 0.0870718
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[244]	training's rmse: 0.0120523	valid_1's rmse: 0.0927594
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[487]	training's rmse: 0.00531875	valid_1's rmse: 0.0989921
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[290]	training's rmse: 0.00900194	valid_1's rmse: 0.0926603
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[774]	training's rmse: 0.00492069	valid_1's rmse: 0.0859028
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[303]	training's rmse: 0.00861226	valid_1's rmse: 0.0926983
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[663]	training's rmse: 0.00406372	valid_1's rmse: 0.0939004

[I 2020-06-21 14:48:19,586] Finished trial#0 with value: 1549.3168175486041 with parameters: {'subsample': 0.4696896142081925, 'subsample_freq': 0.6163672720568971, 'learning_rate': 0.10369559266334792, 'num_leaves': 95, 'feature_fraction': 0.6339909625291371, 'bagging_freq': 1, 'min_child_samples': 6, 'lambda_l2': 1.1747873480659756}. Best is trial#0 with value: 1549.3168175486041.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[306]	training's rmse: 0.0220711	valid_1's rmse: 0.0931172
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[182]	training's rmse: 0.0277761	valid_1's rmse: 0.100125
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[259]	training's rmse: 0.0217136	valid_1's rmse: 0.100493
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[242]	training's rmse: 0.0244973	valid_1's rmse: 0.0944927
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1216]	training's rmse: 0.0106016	valid_1's rmse: 0.0938222
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[143]	training's rmse: 0.0364234	valid_1's rmse: 0.1016
Training until validation scores don't improve for 100 rounds
Early 

...

[I 2020-06-21 15:50:10,203] Finished trial#18 with value: 1458.7316477163706 with parameters: {'subsample': 0.8996533594943326, 'subsample_freq': 0.5069869373497757, 'learning_rate': 0.05091023631368783, 'num_leaves': 70, 'feature_fraction': 0.6039799949164268, 'bagging_freq': 6, 'min_child_samples': 38, 'lambda_l2': 0.1002000537638198}. Best is trial#10 with value: 1396.8995062869105.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	training's rmse: 0.076554	valid_1's rmse: 0.121012
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[116]	training's rmse: 0.0248768	valid_1's rmse: 0.123778
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[26]	training's rmse: 0.0646562	valid_1's rmse: 0.116719
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[35]	training's rmse: 0.0558101	valid_1's rmse: 0.105604
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[28]	training's rmse: 0.0620117	valid_1's rmse: 0.119479
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[29]	training's rmse: 0.0631192	valid_1's rmse: 0.120728
Training until validation scores don't improve for 100 rounds
Early stopping

[44]	training's rmse: 0.0449592	valid_1's rmse: 0.115155
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	training's rmse: 0.0502769	valid_1's rmse: 0.119149
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[429]	training's rmse: 0.00832846	valid_1's rmse: 0.123866
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[31]	training's rmse: 0.0558121	valid_1's rmse: 0.118994
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[39]	training's rmse: 0.0477639	valid_1's rmse: 0.116453
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[248]	training's rmse: 0.0090126	valid_1's rmse: 0.108883
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[42]	training's rmse: 0.0441529	valid_1's rmse: 0.123022
Training until v

[137]	training's rmse: 0.0145224	valid_1's rmse: 0.11377
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[51]	training's rmse: 0.0352925	valid_1's rmse: 0.113316
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	training's rmse: 0.0585267	valid_1's rmse: 0.1138
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[30]	training's rmse: 0.0564743	valid_1's rmse: 0.114525
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[36]	training's rmse: 0.0479307	valid_1's rmse: 0.115588
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[85]	training's rmse: 0.0225554	valid_1's rmse: 0.119253
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[37]	training's rmse: 0.0493563	valid_1's rmse: 0.115825
Training until valida

[I 2020-06-21 15:51:05,659] Finished trial#19 with value: 2109.1901441622417 with parameters: {'subsample': 0.5935142594680825, 'subsample_freq': 0.4158330861636041, 'learning_rate': 0.45829505171905754, 'num_leaves': 88, 'feature_fraction': 0.4016182740418464, 'bagging_freq': 4, 'min_child_samples': 19, 'lambda_l2': 1.9889504746835727}. Best is trial#10 with value: 1396.8995062869105.


In [31]:
print(f"""
Number of finished trials :     {len(study.trials)}
Best value                :  {study.best_value :.3f}
""")

print('Best trial:', study.best_trial.params)


Number of finished trials :     20
Best value                :  1396.900

Best trial: {'subsample': 0.8897026631967412, 'subsample_freq': 0.42708068942389565, 'learning_rate': 0.030223062885783494, 'num_leaves': 66, 'feature_fraction': 0.4294129948533598, 'bagging_freq': 4, 'min_child_samples': 11, 'lambda_l2': 6.563593012634628}


In [7]:
a = len(study.trials)
with open(f'study_{a}times.pickle', 'wb') as f:
    pickle.dump(study, f)